порядок записи ключевых слов выглядит так:

SELECT-
FROM-
WHERE-
GROUP BY-
ORDER BY-
LIMIT

В то же время порядок выполнения операторов в запросе следующий:

Сначала выполняется оператор FROM — происходит выбор нужной таблицы.

Далее WHERE — отфильтровываются строки, соответствующие условию.

Потом GROUP BY — строки объединяются в группы и производится агрегация.

Затем SELECT — отбираются указанные столбцы.

Потом ORDER BY — производится сортировка результирующей таблицы.

И в самом конце LIMIT — ограничивается количество выводимых записей.


**___________________________________________________________________**

К группам, образовавшимся в результате GROUP BY, можно применять сразу несколько агрегирующих функций (в том числе к разным колонкам):

SELECT column_1, SUM(column_2) AS sum, AVG(column_3) AS avg \
FROM table \
GROUP BY column_1

SELECT column_1, COUNT(column_1) AS count\
FROM table\
GROUP BY column_1


**___________________________________________________________________**

После группировки к результатам агрегирующих функций можно применять другие функции. Также над ними можно сразу проводить арифметические операции:

SELECT column_1, ROUND(SUM(column_2), 2) AS rounded_sum\
FROM table\
GROUP BY column_1


SELECT column_1, SUM(column_2)::DECIMAL / SUM(column_3) * 100 AS share\
FROM table\
GROUP BY column_1


**___________________________________________________________________**

Агрегацию не обязательно проводить по уже имеющимся колонкам — можно сначала применить некоторую функцию в колонке из исходной таблицы, а затем сразу провести агрегацию по новой колонке:

SELECT column_1, SUM(some_function(column_2)) AS sum_over_function\
FROM table\
GROUP BY column_1


**___________________________________________________________________**

Группировку можно делать сразу по новым полям, посчитанным в SELECT. При этом допускается использование в GROUP BY алиаса колонки, указанного в SELECT.

Например, если нам необходимо сгруппировать данные по дням, выделив дату из какой-то колонки, и посчитать сумму некоторых значений в каждый из дней, то можно использовать следующие запросы, которые дадут одинаковый результат:

SELECT DATE(column_1) AS date, SUM(column_2) AS sum\
FROM table\
GROUP BY DATE(column_1)


SELECT DATE(column_1) AS date, SUM(column_2) AS sum\
FROM table\
GROUP BY date


**___________________________________________________________________**

 делать агрегацию после группировки не обязательно. Если не указать агрегирующую функцию, то запрос вернёт уникальные значения в столбце (или уникальные комбинации значений в столбцах), т.е. тот же результат, что и оператор DISTINCT. Можете самостоятельно запустить следующие запросы в Redash и убедиться:

SELECT user_id\
FROM user_actions\
GROUP BY user_id


SELECT DISTINCT user_id\
FROM user_actions


**___________________________________________________________________**

при использовании группировки колонки, указанные в SELECT, должны находиться и в GROUP BY, если они не используются в агрегирующих функциях.

Это обязательное условие, и если оно не будет выполнено, то база данных вернёт ошибку.

SELECT column_1, column_2, SUM(column_3) AS sum\
FROM table\
GROUP BY column_1, column_2

Обратите внимание, что в обратную сторону указанное выше правило не работает: если мы что-то указали в GROUP BY, то это совсем не обязательно указывать в SELECT. Иными словами, можно не выводить наименования колонок, по которым осуществляется группировка.


**___________________________________________________________________**

И наконец, группировка всегда выполняется после фильтрации, т.е. сначала выполняются инструкции в WHERE и только потом данные группируются с помощью GROUP BY:

SELECT column_1, SUM(column_2) AS sum\
FROM table\
WHERE column_3 > 0\
GROUP BY column_1

**#1**

Задание:\
С помощью оператора GROUP BY посчитайте количество курьеров мужского и женского пола в таблице couriers.\
Новую колонку с числом курьеров назовите couriers_count.\
Результат отсортируйте по этой колонке по возрастанию.\
Поля в результирующей таблице: sex, couriers_count

SELECT sex,\
       count(sex) as couriers_count\
FROM   couriers\
GROUP BY sex\
ORDER BY couriers_count


**#2**

Задание:\
Посчитайте количество созданных и отменённых заказов в таблице user_actions.

Новую колонку с числом заказов назовите orders_count.

Результат отсортируйте по числу заказов по возрастанию.

Поля в результирующей таблице: action, orders_count

SELECT action,\
       count(action) as orders_count\
FROM   user_actions\
GROUP BY action\
ORDER BY orders_count


**############################################################**

Функция DATE_TRUNC используется для усечения дат и времени, т.е. она работает аналогично округлению ROUND, только для типов данных TIMESTAMP и INTERVAL.

SELECT DATE_TRUNC(part, column)

На месте part в кавычках указывается, до какой точности следует округлять переданное значение времени:  'year', 'month', 'day', 'hour' и т.д.

**############################################################**

**#3**

Задание:\
Используя группировку и функцию DATE_TRUNC, приведите все даты к началу месяца и посчитайте, сколько заказов было сделано в каждом из них.

Расчёты проведите по таблице orders. Колонку с усечённой датой назовите month, колонку с количеством заказов — orders_count.

Результат отсортируйте по месяцам — по возрастанию.

Поля в результирующей таблице: month, orders_count

SELECT date_trunc('month', creation_time) as month,\
       count(order_id) as orders_count\
FROM   orders\
GROUP BY month\
ORDER BY month


**#3**

Задание:\
Используя группировку и функцию DATE_TRUNC, приведите все даты к началу месяца и посчитайте, сколько заказов было сделано и сколько было отменено в каждом из них.

В этот раз расчёты проведите по таблице user_actions. Колонку с усечённой датой назовите month, колонку с количеством заказов — orders_count.

Результат отсортируйте сначала по месяцам — по возрастанию, затем по типу действия — тоже по возрастанию.

Поля в результирующей таблице: month, action, orders_count

SELECT action,\
       date_trunc('month', time) as month,\
       count(order_id) as orders_count\
FROM   user_actions\
GROUP BY month, action\
ORDER BY month, action


**#3**

По данным в таблице users посчитайте максимальный порядковый номер месяца среди всех порядковых номеров месяцев рождения пользователей сервиса. С помощью группировки проведите расчёты отдельно в двух группах — для пользователей мужского и женского пола.

Новую колонку с максимальным порядковым номером месяца рождения в группах назовите max_month. Преобразуйте значения в новой колонке в формат INTEGER, чтобы порядковый номер был выражен целым числом.

Результат отсортируйте по колонке с полом пользователей.

Поля в результирующей таблице: sex, max_month

SELECT sex,\
       max(date_part('month', birth_date))::integer as max_month\
FROM   users\
GROUP BY sex


**#3**

Задание:\
По данным в таблице users посчитайте порядковый номер месяца рождения самого молодого пользователя сервиса. С помощью группировки проведите расчёты отдельно в двух группах — для пользователей мужского и женского пола.

Новую колонку c порядковым номером месяца рождения самого молодого пользователя в группах назовите max_month. Преобразуйте значения в новой колонке в формат INTEGER, чтобы порядковый номер был выражен целым числом.

Результат отсортируйте по колонке с полом пользователей.

Поля в результирующей таблице: sex, max_month

SELECT sex,\
       date_part('month', max(birth_date))::integer as max_month\
FROM   users\
GROUP BY sex


**#3**




**#3**
**#3**